In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MN_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1155487803,MN,"100 N. 6TH ST., MINNEAPOLIS, MN",44.980211,-93.275041,2.705313e+14
1,1747147102,MN,"900 Mendelssohn Ave N 0.0, Golden Valley, MN",44.988779,-93.398813,2.705302e+14
2,1956657109,MN,"315 North Broadway 0.0, New Ulm, MN",44.315885,-94.464125,2.701596e+14
3,2175738706,MN,"553 Fairview Ave N, Saint Paul, MN",44.958093,-93.178051,2.712303e+14
4,3170188800,MN,"801 Main Ave Ste 201, Moorhead, MN",46.873463,-96.767353,2.702702e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1155487803,MN,"100 N. 6TH ST., MINNEAPOLIS, MN",44.980211,-93.275041,2.705313e+14
1,1747147102,MN,"900 Mendelssohn Ave N 0.0, Golden Valley, MN",44.988779,-93.398813,2.705302e+14
2,1956657109,MN,"315 North Broadway 0.0, New Ulm, MN",44.315885,-94.464125,2.701596e+14
3,2175738706,MN,"553 Fairview Ave N, Saint Paul, MN",44.958093,-93.178051,2.712303e+14
4,3170188800,MN,"801 Main Ave Ste 201, Moorhead, MN",46.873463,-96.767353,2.702702e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
683,4751267001,MN,"4990 N. Highway 169, MINNEAPOLIS, MN",39.501209,-94.569531,NaN
757,1787108701,MN,"3596 15th St NE, Oslo, MN",48.193296,-97.145331,NaN
1011,2918528410,MN,"6587 US Highway 75, Wheaton, MN",38.697223,-95.687191,NaN
1012,6719917005,MN,"6587 U.S. Highway 75, WHEATON, MN",38.697223,-95.687191,NaN
1091,1431558706,MN,"4990 Highway 169 N, Minneapolis, MN",39.501209,-94.569531,NaN
2871,1985607104,MN,"3108 Hwy 52 N, ROCHESTER, MN",35.235470,-80.136770,NaN
3921,8343487007,MN,"24500 US HWY 14, LAMBERTON, MN",43.191287,-90.195546,NaN
5634,9308937402,MN,"1554 HIGHWAY 2, TWO HARBORS, MN",44.390085,-72.251419,NaN
5750,7160687000,MN,"4646 Highway 52 North, ROCHESTER, MN",39.540867,-85.355712,NaN
6077,8553667000,MN,"5020 Highway 52 North, ROCHESTER, MN",34.005224,-79.750550,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('27')] #MN
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/34 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

34


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10424/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
683,4751267001,MN,"4990 N. Highway 169, MINNEAPOLIS, MN",39.501209,-94.569531,[]
757,1787108701,MN,"3596 15th St NE, Oslo, MN",48.193296,-97.145331,[]
1011,2918528410,MN,"6587 US Highway 75, Wheaton, MN",38.697223,-95.687191,[]
1012,6719917005,MN,"6587 U.S. Highway 75, WHEATON, MN",38.697223,-95.687191,[]
1091,1431558706,MN,"4990 Highway 169 N, Minneapolis, MN",39.501209,-94.569531,[]
2871,1985607104,MN,"3108 Hwy 52 N, ROCHESTER, MN",35.235470,-80.136770,[]
3921,8343487007,MN,"24500 US HWY 14, LAMBERTON, MN",43.191287,-90.195546,[]
5634,9308937402,MN,"1554 HIGHWAY 2, TWO HARBORS, MN",44.390085,-72.251419,[]
5750,7160687000,MN,"4646 Highway 52 North, ROCHESTER, MN",39.540867,-85.355712,[]
6077,8553667000,MN,"5020 Highway 52 North, ROCHESTER, MN",34.005224,-79.750550,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10424/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
683,4751267001,MN,"4990 N. Highway 169, MINNEAPOLIS, MN",39.501209,-94.569531,NaN
757,1787108701,MN,"3596 15th St NE, Oslo, MN",48.193296,-97.145331,NaN
1011,2918528410,MN,"6587 US Highway 75, Wheaton, MN",38.697223,-95.687191,NaN
1012,6719917005,MN,"6587 U.S. Highway 75, WHEATON, MN",38.697223,-95.687191,NaN
1091,1431558706,MN,"4990 Highway 169 N, Minneapolis, MN",39.501209,-94.569531,NaN
2871,1985607104,MN,"3108 Hwy 52 N, ROCHESTER, MN",35.235470,-80.136770,NaN
3921,8343487007,MN,"24500 US HWY 14, LAMBERTON, MN",43.191287,-90.195546,NaN
5634,9308937402,MN,"1554 HIGHWAY 2, TWO HARBORS, MN",44.390085,-72.251419,NaN
5750,7160687000,MN,"4646 Highway 52 North, ROCHESTER, MN",39.540867,-85.355712,NaN
6077,8553667000,MN,"5020 Highway 52 North, ROCHESTER, MN",34.005224,-79.750550,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
683,4751267001,MN,"4990 N. Highway 169, MINNEAPOLIS, MN",39.501209,-94.569531,NaN
757,1787108701,MN,"3596 15th St NE, Oslo, MN",48.193296,-97.145331,NaN
1011,2918528410,MN,"6587 US Highway 75, Wheaton, MN",38.697223,-95.687191,NaN
1012,6719917005,MN,"6587 U.S. Highway 75, WHEATON, MN",38.697223,-95.687191,NaN
1091,1431558706,MN,"4990 Highway 169 N, Minneapolis, MN",39.501209,-94.569531,NaN
2871,1985607104,MN,"3108 Hwy 52 N, ROCHESTER, MN",35.235470,-80.136770,NaN
3921,8343487007,MN,"24500 US HWY 14, LAMBERTON, MN",43.191287,-90.195546,NaN
5634,9308937402,MN,"1554 HIGHWAY 2, TWO HARBORS, MN",44.390085,-72.251419,NaN
5750,7160687000,MN,"4646 Highway 52 North, ROCHESTER, MN",39.540867,-85.355712,NaN
6077,8553667000,MN,"5020 Highway 52 North, ROCHESTER, MN",34.005224,-79.750550,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
683,4751267001,NaN
757,1787108701,NaN
1011,2918528410,NaN
1012,6719917005,NaN
1091,1431558706,NaN
2871,1985607104,NaN
3921,8343487007,NaN
5634,9308937402,NaN
5750,7160687000,NaN
6077,8553667000,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/MN_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,476654,1155487803,MN,"100 N. 6TH ST., MINNEAPOLIS, MN",44.980211,-93.275041,2.705313e+14
1,476655,1747147102,MN,"900 Mendelssohn Ave N 0.0, Golden Valley, MN",44.988779,-93.398813,2.705302e+14
2,476656,1956657109,MN,"315 North Broadway 0.0, New Ulm, MN",44.315885,-94.464125,2.701596e+14
3,476657,2175738706,MN,"553 Fairview Ave N, Saint Paul, MN",44.958093,-93.178051,2.712303e+14
4,476658,3170188800,MN,"801 Main Ave Ste 201, Moorhead, MN",46.873463,-96.767353,2.702702e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,476654,1155487803,MN,"100 N. 6TH ST., MINNEAPOLIS, MN",44.980211,-93.275041,2.705313e+14,NaN
1,476655,1747147102,MN,"900 Mendelssohn Ave N 0.0, Golden Valley, MN",44.988779,-93.398813,2.705302e+14,NaN
2,476656,1956657109,MN,"315 North Broadway 0.0, New Ulm, MN",44.315885,-94.464125,2.701596e+14,NaN
3,476657,2175738706,MN,"553 Fairview Ave N, Saint Paul, MN",44.958093,-93.178051,2.712303e+14,NaN
4,476658,3170188800,MN,"801 Main Ave Ste 201, Moorhead, MN",46.873463,-96.767353,2.702702e+14,NaN
...,...,...,...,...,...,...,...,...
19715,496369,9536197309,MN,"1532 NORTHDALE BLVD NW, COON RAPIDS, MN",45.182172,-93.305332,2.700305e+14,NaN
19716,496370,9584877110,MN,"618 W 41ST ST, HIBBING, MN",47.399596,-92.949461,2.713701e+14,NaN
19717,496371,9660767104,MN,"75 W viking Drive #104, SAINT PAUL, MN",45.008600,-93.103767,2.712304e+14,NaN
19718,496372,9769477006,MN,"9188 COTTONWOOD LN N, MAPLE GROVE, MN",45.121334,-93.423942,2.705303e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,476654,1155487803,MN,"100 N. 6TH ST., MINNEAPOLIS, MN",44.980211,-93.275041,2.705313e+14,NaN,270531261025007.0
1,476655,1747147102,MN,"900 Mendelssohn Ave N 0.0, Golden Valley, MN",44.988779,-93.398813,2.705302e+14,NaN,270530216012007.0
2,476656,1956657109,MN,"315 North Broadway 0.0, New Ulm, MN",44.315885,-94.464125,2.701596e+14,NaN,270159603003018.0
3,476657,2175738706,MN,"553 Fairview Ave N, Saint Paul, MN",44.958093,-93.178051,2.712303e+14,NaN,271230319001043.0
4,476658,3170188800,MN,"801 Main Ave Ste 201, Moorhead, MN",46.873463,-96.767353,2.702702e+14,NaN,270270203002037.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1155487803,MN,"100 N. 6TH ST., MINNEAPOLIS, MN",44.980211,-93.275041,270531261025007.0
1,1747147102,MN,"900 Mendelssohn Ave N 0.0, Golden Valley, MN",44.988779,-93.398813,270530216012007.0
2,1956657109,MN,"315 North Broadway 0.0, New Ulm, MN",44.315885,-94.464125,270159603003018.0
3,2175738706,MN,"553 Fairview Ave N, Saint Paul, MN",44.958093,-93.178051,271230319001043.0
4,3170188800,MN,"801 Main Ave Ste 201, Moorhead, MN",46.873463,-96.767353,270270203002037.0
...,...,...,...,...,...,...
19715,9536197309,MN,"1532 NORTHDALE BLVD NW, COON RAPIDS, MN",45.182172,-93.305332,270030507041002.0
19716,9584877110,MN,"618 W 41ST ST, HIBBING, MN",47.399596,-92.949461,271370121002035.0
19717,9660767104,MN,"75 W viking Drive #104, SAINT PAUL, MN",45.008600,-93.103767,271230421013012.0
19718,9769477006,MN,"9188 COTTONWOOD LN N, MAPLE GROVE, MN",45.121334,-93.423942,270530267261002.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
683,4751267001,MN,"4990 N. Highway 169, MINNEAPOLIS, MN",39.501209,-94.569531,NaN
757,1787108701,MN,"3596 15th St NE, Oslo, MN",48.193296,-97.145331,NaN
1011,2918528410,MN,"6587 US Highway 75, Wheaton, MN",38.697223,-95.687191,NaN
1012,6719917005,MN,"6587 U.S. Highway 75, WHEATON, MN",38.697223,-95.687191,NaN
1091,1431558706,MN,"4990 Highway 169 N, Minneapolis, MN",39.501209,-94.569531,NaN
2871,1985607104,MN,"3108 Hwy 52 N, ROCHESTER, MN",35.235470,-80.136770,NaN
3921,8343487007,MN,"24500 US HWY 14, LAMBERTON, MN",43.191287,-90.195546,NaN
5634,9308937402,MN,"1554 HIGHWAY 2, TWO HARBORS, MN",44.390085,-72.251419,NaN
5750,7160687000,MN,"4646 Highway 52 North, ROCHESTER, MN",39.540867,-85.355712,NaN
6077,8553667000,MN,"5020 Highway 52 North, ROCHESTER, MN",34.005224,-79.750550,NaN


Convert to File

In [19]:
fips_MN = fips_merge_drop

In [20]:
fips_MN.to_csv("../../../data/state_data/geo/geo_fips/23/MN_fips_scraped.csv") 